In [1]:
import numpy as np
import pandas as pd
import cv2
import os as os
from random import shuffle
from tqdm import tqdm


In [2]:
img_size = 50
lr = 1e-3
train_dir = 'C:\\dogs-vs-cats\\train'
test_dir = 'C:\\dogs-vs-cats\\test'


In [3]:
model_name="dogsvscats-{}-{}.model".format(lr,'6conv-basic-video')

In [4]:
def label_img(img):
    word_label=img.split('.')[-3]
    if word_label=='cat' : return [1,0]
    elif word_label=='dog' : return [0,1]
    

In [5]:
def create_train_data(): 
    training_data=[]
   
    for img in os.listdir(train_dir):
        label=label_img(img)
        path=os.path.join(train_dir,img)
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(img_size,img_size))
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data
        

In [ ]:
create_train_data()

In [ ]:
def process_test_data():
    testing_data=[]
    for img in os.listdir(test_dir):
        img_num=img.split('.')[0]
        path=os.path.join(test_dir,img)
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(img_size,img_size))
        testing_data.append([np.array(img),img_num])
        
    np.save('test_data.npy',testing_data)
    return testing_data

    

In [ ]:
train_data=np.load('train_data.npy')

In [ ]:
print(train_data.shape)

In [ ]:
train_data[0]

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensrflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, img_size, img_size, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)


convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)


convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)


convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [ ]:
if os.path.exists('{}.meta'.format(model_name)):
    model.load(model_name)
    print('Model loaded!')

In [ ]:
train=train_data[:-500]
test=train_data[-500:]

X=np.array([i[0] for i in train]).reshape(-1,img_size,img_size,1)
Y=[i[1] for i in train]

test_x=np.array([i[0] for i in test]).reshape(-1,img_size,img_size,1)
test_y=[i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=model_name)

In [ ]:
 tensorboard --logdir=foo:"C:\Users\saish.mohare\log" 
        
        